In [1]:
import gzip
from collections import defaultdict
import sklearn
from sklearn import linear_model
import string
import re
import datetime, time
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as md
import math
from sklearn import svm

In [2]:
#####
# Setup
#####

In [2]:
def readCSV(path):
    f = open(path, 'rt')
    f.readline()

    for l in f:
        
        if l.startswith("timestamp"): 
            continue

        try: 
            timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education = re.split(r',(?![ ])', l)

        except:
            continue

        yield timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education

In [3]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [5]:
#####
# Parse data from dataset
#####

In [4]:
# id , unixtime, company, level, title, total_comp, city, state, experience, tenure, tag, 
# salary, stock, bonus, city_id, dma_id, ms_deg, bs_deg, phd_deg, hs, college
def parseSalaryRaw(salaryRaw):
    salaryAllData = []

    for entry in salaryRaw:

        date, clocktime = entry[0].split(' ')
        date = date.split('/')
        clocktime = clocktime.split(':')
        timestamp = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(clocktime[0]), int(clocktime[1]), int(clocktime[2]))
        timestamp = int(time.mktime(timestamp.timetuple()))


        company = entry[1]
        level = entry[2]
        title = entry[3]

        total_comp = int(float(entry[4]))

        location = entry[5]
        if location.count(',') == 2:
            city, state, country = location.strip('"').split(', ')
        elif location.count(',') == 1:
            city, state = location.strip('"').split(', ')
            country = 'USA'

        experience = entry[6]
        tenure = entry[7]

        tag = entry[8]

        salary = int(float(entry[9]))
        if salary == 0:
            continue

        stock = int(float(entry[10]))
        bonus = int(float(entry[11]))

        g = entry[12]
        gender = -1
        if g == 'Female':
            gender = 0
        elif g == 'Male':
            gender = 1
        elif g == 'Other':
            gender = 2

        city_id = int(entry[14])

        try: 
            dma_id = int(entry[15])
        except:
            dma_id = -1

        id = int(entry[16])

        ms_deg = int(entry[17])
        bs_deg = int(entry[18])
        phd_deg = int(entry[18])
        hs = int(entry[19])
        college = int(entry[20])

        r = entry[27]
        race = -1
        if r == 'Asian':
            race = 0
        elif r == 'Black':
            race = 1
        elif r == 'Hispanic':
            race = 2
        elif r == 'Two Or More':
            race = 3
        elif r == 'White':
            race = 4


        salaryAllData.append({
            'id': id,
            'timestamp': timestamp,
            'company': company,
            'level': level,
            'title': title,
            'total_comp': total_comp,
            'city': city,
            'state': state,
            'country': country,
            'experience': experience,
            'tenure': tenure,
            'tag': tag,
            'salary': salary,
            'stock': stock,
            'bonus': bonus,
            'gender': gender,
            'city_id': city_id,
            'dma_id': dma_id,
            'ms_deg': ms_deg,
            'bs_deg': bs_deg,
            'phd_deg': phd_deg,
            'hs': hs,
            'college': college,
            'race': race,
        })
    return salaryAllData


In [5]:
# Reparse data from CSV so we don't mess up above data
salaryRaw = []
for l in readCSV("Levels_Fyi_Salary_Data.csv"):
    salaryRaw.append(l)

salaryAllData = parseSalaryRaw(salaryRaw)
salaryAllData[0]['id']

1

In [6]:
# Parse raw data
salaryAllData = parseSalaryRaw(salaryRaw)

In [7]:
#####
# Training, Validation, and Test sets
#####

# shuffle data
shuffledAllData = sklearn.utils.shuffle(salaryAllData)
shuffledAllData[0]['id']

# create 80 / 10 / 10 : Train / Validation / Test sets
X = shuffledAllData
y = [d['salary'] for d in shuffledAllData]

X_train, X_temp, y_train, y_temp = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = sklearn.model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Validate that sets were created with propper sizes
print(str(len(X_train)) + ", " + str(len(y_train)))
print(str(len(X_valid)) + ", " + str(len(y_valid)))
print(str(len(X_test)) + ", " + str(len(y_test)))

# Validate all ids are different
print(str(X_train[0]['id']) + ", " + str(X_valid[0]['id']) + ", " + str(X_test[0]['id']))

47996, 47996
6000, 6000
6000, 6000
74607, 14433, 41630


In [10]:
#####
# Baselines
#   1. Predict average salary in dataset
#   2. Predict average salary for given Years of Experience
#####

In [8]:
# Create some useful collections for these baseline

# create YOE to Average Map based on training data
# We create two dictionaries to track the total salary seen at each YOE
salByExpMap = defaultdict(int)

# And the number of salaries seen at that YOE
entriesByExpMap = defaultdict(int)

# For each entry increment the total sal and num entries at that YOE
# (This could be done with a running avg technique but that is too much work to code)
for x in X_train:
    exp, sal = float(x['experience']), float(x['salary'])
    exp = math.ceil(exp)
    
    total = 0
    numEntries = 0
    
    if salByExpMap[exp]:
        total = salByExpMap[exp]
    if entriesByExpMap[exp]:
        numEntries = entriesByExpMap[exp]
    
    total += sal
    numEntries += 1
    
    salByExpMap[exp] = total
    entriesByExpMap[exp] = numEntries
    

    
# Calculate averages by YOE as well as total avg
totalSal = 0
totalCount = 0

averageSalaryByYoeMap = defaultdict(int)
for exp in entriesByExpMap:
    avg = salByExpMap[exp] / entriesByExpMap[exp]
    averageSalaryByYoeMap[exp] = avg
    
    totalSal += salByExpMap[exp]
    totalCount += entriesByExpMap[exp]

avgSal = totalSal / totalCount

print("Average salary: " + str(avgSal))


def pred_baseline1(x):    
    return avgSal
    
def pred_baseline2(x):    
    exp = math.ceil(float(x['experience']))

    if averageSalaryByYoeMap[exp]:
        return averageSalaryByYoeMap[exp]
    else:
        return avgSal

Average salary: 142026.25741728477


In [9]:
def baseline1():
    # Returns average salary in dataset
    y_b1 = []
    y_b1_pred = []
    
    avg_err = 0
    avg_percent_err = 0
    
    for d in X_valid:
        pred = pred_baseline1(d)
        actual = d['salary']
        
        y_b1_pred.append(pred)
        y_b1.append(actual)
        
        avg_err += abs(actual - pred)
        avg_percent_err += abs(100 * ((pred - actual) / actual))
        
    mse = MSE(y_b1_pred, y_b1)
    
    avg_err = avg_err / len(X_valid)
    avg_percent_err = avg_percent_err / len(X_valid)
    
    return mse, avg_err, avg_percent_err

In [10]:
mse_b1, avg_err_b1, avg_percent_err_b1 = baseline1()
print(mse_b1)
print(avg_err_b1)
print(avg_percent_err_b1)

3110427580.315391
39493.53048836722
47.49972223352079


In [11]:
def baseline2():
    # Takes rounded YOE for each datapoint and predicts average salary for that YOE
    # If there are no entries for that YOE, predict overall average
    y_b1 = []
    y_b1_pred = []
    
    avg_err = 0
    avg_percent_err = 0
    
    for d in X_valid:
        pred = pred_baseline2(d)
        actual = d['salary']
        
        y_b1_pred.append(pred)
        y_b1.append(actual)
        
        avg_err += abs(actual - pred)
        avg_percent_err += abs(100 * ((pred - actual) / actual))
        
    mse = MSE(y_b1_pred, y_b1)
    
    avg_err = avg_err / len(X_valid)
    avg_percent_err = avg_percent_err / len(X_valid)
    
    return mse, avg_err, avg_percent_err

In [12]:
mse_b2, avg_err_b2, avg_percent_err_b2 = baseline2()
print(mse_b2)
print(avg_err_b2)
print(avg_percent_err_b2)

2454571515.1482697
34081.43203070285
41.11578914323483


In [16]:
#####
# Model
#####

In [17]:
#####
# Provide a description of your model:
#      1. What model are you using
#      2. What information are you trying to incorporate into your model
#      3. How are you building your feature vector?
#      4. What interesting pieces of information did you learn? (i.e. most influential words, popular cities, etc)
#####

# Write description in comments here

In [18]:
# Model code starts here

In [ ]:
# Simple sliding window

In [7]:
combined = []

for d in salaryAllData:
    combined.append((d['timestamp'], d['salary']))

combined.sort()
print(combined[:10])
d = combined[len(combined)-1][0]
print(datetime.datetime.fromtimestamp(d))

[(1496860407, 107000), (1497218037, 155000), (1497500545, 169000), (1497635041, 120000), (1497684194, 157000), (1498009799, 110000), (1498160271, 180000), (1498164926, 135000), (1498192788, 165000), (1498537545, 157000)]
2021-08-17 08:28:57


In [8]:
X_time = [d[0] for d in combined]
y_time = [d[1] for d in combined]

In [9]:
wSize = 1000
ySum = sum(y_time[:wSize])
sliding = []

print(ySum)

for i in range(0, wSize):
    sliding.append(ySum/wSize)

for i in range(wSize,len(y_time)):
    ySum += y_time[i-1] - y_time[i-wSize-1]
    sliding.append(ySum*1.0/wSize)


print(len(sliding))
print(len(y_time))
print(y_time[10])
print(sliding[10])

154885243
59996
59996
110000
154885.243


In [10]:
print(sliding[0])
print(y_time[0])

154885.243
107000


In [13]:
mse = MSE(sliding, y_time)
mae = sklearn.metrics.mean_absolute_error(y_time, sliding)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_time, sliding)

In [14]:
print(mse)
print(mae)
print(mape)

3141751781.0261955
39534.61891631109
47.382833306299716


In [ ]:
# sliding window, disregarding most receent datapoint

In [27]:
wSize = 500
ySum = sum(y_time[:wSize])
sliding = []

print(ySum)

for i in range(0, wSize):
    sliding.append(ySum/wSize)

    
alpha = 0

for i in range(wSize,len(y_time)):

    ySum = sum(sliding[i-wSize:i-1])
    sliding.append(ySum*1.0/wSize)


print(len(sliding))
print(len(y_time))
print(y_time[20])
print(sliding[20])

77811073
59996
59996
150000
155622.146


In [28]:
mse = MSE(sliding, y_time)

avg_err = 0
avg_percent_err = 0

mae = sklearn.metrics.mean_absolute_error(y_time, sliding)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_time, sliding)

In [29]:
print(mse)
print(mae)
print(mape)

5316796018.690831
54902.67770759424
44.522933989944306


In [ ]:
# One-hot encodings

In [30]:
minYear = 2017
maxYear = 2021

def feature_time(d, inc_year, inc_month, inc_day, inc_hour):
    
    unix_time = d['timestamp']
    date = datetime.datetime.fromtimestamp(unix_time)

    year = date.year
    month = date.month
    day = date.day
    hour = date.hour
    minute = date.minute

    one_hot_year = [0]*(maxYear-minYear)
    y_index = year - minYear

    if y_index != 0:
        one_hot_year[y_index-1] = 1

    one_hot_month = [0]*11
    m_index = month - 1

    if m_index != 0:
        one_hot_month[m_index-1] = 1

    one_hot_day = [0]*31
    d_index = day - 1

    if d_index != 0:
        one_hot_day[d_index-1] = 1

    one_hot_hour = [0]*23
    h_index = hour - 1

    if h_index != 0:
        one_hot_hour[h_index-1] = 1

    yoe = float(d['experience'])
    feat = []
    # feat = [yoe]


    if inc_year:
        feat += one_hot_year
    if inc_month:
        feat += one_hot_month
    if inc_day:
        feat += one_hot_day
    if inc_hour:
        feat += one_hot_hour

    # feat.append(yoe)
            
    return feat

In [ ]:
# different combinations of year/month/day/hour

In [31]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, True, True, True, True) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

pred = mod.predict(X_train_time)

mse = sklearn.metrics.mean_squared_error(y_train_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_train_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_train_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

55180.4321205081
38974.127589757736
45.810831423485986


In [48]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, True, False, True, True) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

pred = mod.predict(X_train_time)

mse = sklearn.metrics.mean_squared_error(y_train_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_train_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_train_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

3074023494.1354356
39066.98439182126
46.06996102818901


In [49]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, True, True, False, True) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

pred = mod.predict(X_train_time)

mse = sklearn.metrics.mean_squared_error(y_train_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_train_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_train_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

3064074377.596676
38991.77881363036
45.8927103168844


In [50]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, True, True, True, False) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

pred = mod.predict(X_train_time)

mse = sklearn.metrics.mean_squared_error(y_train_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_train_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_train_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

3135646999.952729
39435.628956404464
47.316368949502966


In [ ]:
# best combo for 2 true

In [67]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, False, False, True, True) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

pred = mod.predict(X_train_time)

mse = sklearn.metrics.mean_squared_error(y_train_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_train_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_train_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

3093990327.9200253
39098.629820731054
46.38646478755724


In [ ]:
# best combo for only 1 true

In [76]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, False, False, False, True) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

pred = mod.predict(X_train_time)

mse = sklearn.metrics.mean_squared_error(y_train_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_train_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_train_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

3096393152.6101418
39106.191810163895
46.40264067517482


In [ ]:
# choose best average error combo on training data to use on valid/test set

In [77]:
mod = linear_model.LinearRegression()

X_train_time = [feature_time(d, True, True, True, True) for d in X_train]
y_train_time = y_train[:]

mod.fit(X_train_time, y_train_time)

LinearRegression()

In [78]:
X_valid_time = [feature_time(d, True, True, True, True) for d in X_valid]
y_valid_time = y_valid[:]

pred = mod.predict(X_valid_time)

mse = sklearn.metrics.mean_squared_error(y_valid_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_valid_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_valid_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

2936734110.8041167
38877.72085695478
45.90300031524025


In [79]:
X_test_time = [feature_time(d, True, True, True, True) for d in X_test]
y_test_time = y_test[:]

pred = mod.predict(X_test_time)

mse = sklearn.metrics.mean_squared_error(y_test_time, pred)
mae = sklearn.metrics.mean_absolute_error(y_test_time, pred)
mape = 100*sklearn.metrics.mean_absolute_percentage_error(y_test_time, pred)

print(math.sqrt(mse))
print(mae)
print(mape)

3181160416.7044244
39826.80607111622
46.495579306841606
